In [1]:
import mne

file = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/MEG/raw_meg/sub-01/ses-1/meg/sub-01_ses-1_task-action_run-01_meg.ds'
raw = mne.io.read_raw_ctf(file)

ds directory : /nfs/z1/userhome/ZhouMing/workingdir/BIN/action/MEG/raw_meg/sub-01/ses-1/meg/sub-01_ses-1_task-action_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       6.03   88.03    0.00 mm <->    6.03   88.03    0.00 mm (orig :  -52.08   68.00 -284.59 mm) diff =    0.000 mm
      -6.03  -88.03    0.00 mm <->   -6.03  -88.03    0.00 mm (orig :   61.15  -67.21 -290.74 mm) diff =    0.000 mm
     103.73    0.00    0.00 mm <->  103.73    0.00    0.00 mm (orig :   73.68   65.67 -246.22 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /nfs/z1/userhome/ZhouMing/workingdir/BIN/action/MEG/raw_meg/sub-01/ses-1/meg/sub-01_ses-1_task-action_run-01_meg.ds/sub-01_ses-1_task-action_run-01_meg.meg4: 
    System clock channel is available, 

In [2]:
raw.info

<Info | 16 non-empty values
 bads: []
 ch_names: SCLK01-177, BG1-4504, BG2-4504, BG3-4504, BP1-4504, BP2-4504, ...
 chs: 108 misc, 28 Reference Magnetometers, 273 Magnetometers
 comps: 5 items (list)
 ctf_head_t: CTF/4D/KIT head -> head transform
 custom_ref_applied: False
 dev_ctf_t: MEG device -> CTF/4D/KIT head transform
 dev_head_t: MEG device -> head transform
 dig: 3 items (3 Cardinal)
 experimenter: xyfeng
 highpass: 0.0 Hz
 hpi_results: 1 item (list)
 lowpass: 600.0 Hz
 meas_date: 2021-11-18 14:44:00 UTC
 meas_id: 4 items (dict)
 nchan: 409
 projs: []
 sfreq: 1200.0 Hz
 subject_info: 1 item (dict)
>

In [2]:
import os
import json
import subprocess
import numpy as np
import pandas as pd
import pickle as pkl
import nibabel as nib
import scipy.io as sio
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from os.path import join as pjoin
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from nilearn.glm.first_level import make_first_level_design_matrix

mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams.update({'font.size': 12, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})

def save_ciftifile(data, filename):
    template = '/nfs/z1/zhenlab/BrainImageNet/NaturalObject/data/bold/Analysis_derivatives/ciftify/sub-core02/MNINonLinear/Results/ses-ImageNet01_task-object_run-1/ses-ImageNet01_task-object_run-1_Atlas.dtseries.nii'
    ex_cii = nib.load(template)
    ex_cii.header.get_index_map(0).number_of_series_points = data.shape[0]
    nib.save(nib.Cifti2Image(data.astype(np.float32), ex_cii.header), filename)

# define path
beta_path = '/nfs/z1/zhenlab/BrainImageNet/action/data/bold/derivatives/beta'
fmriprep_path = '/nfs/z1/zhenlab/BrainImageNet/action/data/bold/derivatives/fmriprep'
ciftify_path = '/nfs/z1/zhenlab/BrainImageNet/action/data/bold/derivatives/ciftify'
nifti_path = '/nfs/z1/zhenlab/BrainImageNet/action/data/bold/nifti'
result_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/utils/data_paper/result'

In [3]:
from scipy.stats import pearsonr

# Load beta for 30 subjects 
sub_names = sorted([i for i in os.listdir(beta_path) if i.startswith('sub')])
mask_sum = ['EVC', 'VTC', 'LO', 'AIP']
n_sub = len(sub_names)
n_class = 180

beta_sum = np.zeros((n_sub, n_class, 59412))
for sub_idx, sub_name in enumerate(sub_names):
    # define beta path
    beta_sub_path = pjoin(beta_path, sub_name, f'{sub_name}_action-beta_clean.npy')
    beta_sub = np.load(beta_sub_path)
    scaler = StandardScaler()
    beta_sum[sub_idx] = scaler.fit_transform(beta_sub)

beta_sum = beta_sum.mean(axis=0)

In [4]:
# load animacy labels
beh_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/utils/action_spaces/behavior'
beh_df = pd.read_csv(pjoin(beh_path, 'dim_scores.csv'), index_col=0)
beh_df['mean_score'] = beh_df.iloc[:, 2:].mean(axis=1)
beh_df = beh_df.drop(beh_df.columns[2:22], axis=1)
beh_df = beh_df.sort_values(by='class')

In [15]:
dims = ['transitivity', 'sociality']
brain_map = np.zeros((2, beta_sum.shape[1]))
for dim_idx, dim in enumerate(dims):
    # get label
    dim_score = beh_df.loc[beh_df['dim']==dim, 'mean_score'].to_numpy()
    dim_content = np.array([1 if item > 4 else 0 for item in dim_score])
    # compute brain map
    brain_map = np.zeros((1, 91282))
    brain_map[0, :59412] = beta_sum[dim_content==1].mean(axis=0) - beta_sum[dim_content==0].mean(axis=0)
    save_ciftifile(brain_map, pjoin(result_path, f'{dim}_map.dtseries.nii'))
    print('%s map threshold: left: %.2f; right: %.2f'%(dim, brain_map.squeeze()[:59412].mean() - brain_map.squeeze()[:59412].std(),
                                                       brain_map.squeeze()[:59412].mean() + brain_map.squeeze()[:59412].std()))


transitivity map threshold: left: -0.08; right: 0.09
sociality map threshold: left: -0.05; right: 0.06
